In [3]:
import requests
from tqdm import tqdm_notebook


In [4]:
API_KEY = "RGAPI-ba834a44-f556-4451-adac-88b09fe10f77"

In [5]:
def make_url_for_summonerinfo_with_summonername(summoner_name):
    return f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={API_KEY}"
def make_url_for_matches_with_gameId(match_id):
    return f"https://kr.api.riotgames.com/lol/match/v4/matches/{match_id}?api_key={API_KEY}"
def make_url_for_matchlist_with_accountId(accountId): # modified for endIndex 2
    return f"https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}?endIndex=2&api_key={API_KEY}"


    

In [6]:
def get_summoner_id(summoner_name):
    A = requests.get(make_url_for_summonerinfo_with_summonername(summoner_name)) # A is summonerinfo
    return A.json()['id']
    

In [7]:
def get_account_id(summoner_name):
    A = requests.get(make_url_for_summonerinfo_with_summonername(summoner_name))
    return A.json()['accountId']

In [8]:
def get_gameIdlist_by_summonername(summoner_name):
    account_id = get_account_id(summoner_name)
    url = make_url_for_matchlist_with_accountId(account_id)
    J = requests.get(url).json() # matchlist from API
    gameId_list = [ x['gameId'] for x in J['matches'] ]
    return gameId_list
    

In [9]:
def get_matchinfo_by_gameId(game_id): #sample. matchlist is a []
    url = make_url_for_matches_with_gameId(game_id)
    J = requests.get(url).json()
    return J # matchDto type
#      match_ids
#      match_infos = [get_match_info_by_matchid(matchid) 
#                     for matchid in matchids]


In [10]:
def get_participantId_from_matchDto(matchDto, summoner_name): #takes a single matchDto, not a list of it
        identities = matchDto['participantIdentities'] # List[{} x 10]
        for x in identities:
            if x['player']['summonerName'].lower() == summoner_name.lower():
                return int(x['participantId']) #returns an integer 1~10       

In [11]:
def get_matchdata_by_summonername(summoner_name): # this is the main function for getting player raw stats
    matchlist = get_gameIdlist_by_summonername(summoner_name) #List[gameId]
    matchinfos = [get_matchinfo_by_gameId(x) 
                  for x in tqdm_notebook(matchlist)] # List[matchDto] type
    correctIds = [get_participantId_from_matchDto(x, summoner_name) for x in matchinfos] # List[int] contains correct ids in order
    pair = zip(matchinfos, correctIds) # a tuple of match and its correctId
    stats = [ matchDto['participants'][int(correctId)-1]['stats'] for (matchDto,correctId) in pair]
    
    return stats
    

In [12]:
def get_kda_from_stats(stats):
    kill = sum([x['kills'] for x in stats])
    death = sum([x['deaths'] for x in stats])
    assists = sum([x['assists'] for x in stats])
    
    return (kill+death)/assists

# add more functions

In [13]:
def get_player_statistics_by_summonername(summoner_name, option):
    stats = get_matchdata_by_summonername(summoner_name)
    
    if option.lower() == 'kda':
        return get_kda_from_stats(stats)
    
    # add more functions

In [14]:
get_player_statistics_by_summonername("iro kkyul", 'kda')

1.0666666666666667

In [15]:
get_matchdata_by_summonername('놀지말고공부하자')

[{'participantId': 7,
  'win': False,
  'item0': 3069,
  'item1': 2420,
  'item2': 1052,
  'item3': 3117,
  'item4': 1006,
  'item5': 2055,
  'item6': 3364,
  'kills': 1,
  'deaths': 5,
  'assists': 3,
  'largestKillingSpree': 0,
  'largestMultiKill': 1,
  'killingSprees': 0,
  'longestTimeSpentLiving': 460,
  'doubleKills': 0,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 15588,
  'magicDamageDealt': 10896,
  'physicalDamageDealt': 1316,
  'trueDamageDealt': 3376,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 3790,
  'magicDamageDealtToChampions': 3259,
  'physicalDamageDealtToChampions': 275,
  'trueDamageDealtToChampions': 256,
  'totalHeal': 533,
  'totalUnitsHealed': 1,
  'damageSelfMitigated': 4016,
  'damageDealtToObjectives': 0,
  'damageDealtToTurrets': 0,
  'visionScore': 18,
  'timeCCingOthers': 54,
  'totalDamageTaken': 6939,
  'magicalDamageTaken': 1021,
  'physicalDamageTaken': 5778,
  'trueDamageTake

In [16]:
def get_tier_by_summonerId(summonerid):
    url = f"https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/{summonerid}?api_key={API_KEY}"
    J = requests.get(url).json()# tier etc text
    return [J[0]['tier'], J[0]['rank']]

def get_tier_by_summonername(summoner_name):
    account_id = get_summoner_id(summoner_name) # 이름통일필요
    tier = get_tier_by_summonerId(account_id)  
    return tier

In [17]:
get_tier_by_summonerId("iUvqYWZL47XbrkiyYs3GAX4nyoYIejeA352lJQf2FDtiZg")

['PLATINUM', 'I']

In [18]:
get_tier_by_summonername("iro kkyul")

['GOLD', 'III']

In [19]:
get_account_id("de4d presidents")

KeyError: 'accountId'

In [ ]:
temp =         {
            "lane": "BOTTOM",
            "gameId": 3540135699,
            "champion": 82,
            "platformId": "KR",
            "timestamp": 1549899182280,
            "queue": 440,
            "role": "DUO_CARRY",
            "season": 11
        }

In [ ]:
get_matchinfo_by_gameId(3540135699)

In [ ]:
get_account_id("de4d presidents")